In [6]:
import requests
import json
import pandas as pd
from tqdm import tqdm
import time

In [7]:
def get_gungu_info(sido_code):
    down_url = 'https://new.land.naver.com/api/regions/list?cortarNo='+sido_code
    r = requests.get(down_url,data={"sameAddressGroup":"false"},headers={
        "Accept-Encoding": "gzip",
        "Host": "new.land.naver.com",
        "Referer": "https://new.land.naver.com/complexes?ms=37.488344,127.04512050000001,17&a=PRE:APT:OPST&b=A1&e=RETAIL",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
    })
    r.encoding = "utf-8-sig"
    temp=json.loads(r.text)
    temp_df = pd.DataFrame(temp['regionList'])

    #temp=list(pd.DataFrame(temp['regionList'])["cortarNo"])
    return temp_df[['cortarNo','cortarName']]

In [8]:
def get_dong_info(gungu_code):
    down_url = 'https://new.land.naver.com/api/regions/list?cortarNo='+gungu_code
    r = requests.get(down_url,data={"sameAddressGroup":"false"},headers={
        "Accept-Encoding": "gzip",
        "Host": "new.land.naver.com",
        "Referer": "https://new.land.naver.com/complexes?ms=37.488344,127.04512050000001,17&a=PRE:APT:OPST&b=A1&e=RETAIL",
        "Sec-Fetch-Dest": "empty",
        "Sec-Fetch-Mode": "cors",
        "Sec-Fetch-Site": "same-origin",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36"
    })
    r.encoding = "utf-8-sig"
    temp=json.loads(r.text)
    temp_df = pd.DataFrame(temp['regionList'])
    #temp=list(pd.DataFrame(temp['regionList'])["cortarNo"])
    return temp_df[['cortarNo','cortarName','centerLat','centerLon']]

In [9]:
import warnings, os, time, shutil, urllib

warnings.filterwarnings(action='ignore')
# 파이썬에서 불필요한 warning은 안뜨게 함

from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

options = webdriver.ChromeOptions()
options.add_argument("--ignore-local-proxy")

try:
    driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()), options=options)
except:
    driver = webdriver.Chrome()


In [10]:
gungu_result = get_gungu_info('1100000000')
gu_cortarNo = gungu_result['cortarNo']
gu_cortarName = gungu_result['cortarName']
total_df = pd.DataFrame()

for i in tqdm(range(len(gungu_result))):
    dong_result = get_dong_info(gu_cortarNo[i])
    dong_cortarNo = dong_result['cortarNo']
    dong_cortarName = dong_result['cortarName']
    dong_centerLat = dong_result['centerLat']
    dong_centerLon = dong_result['centerLon']
    pre_gu_cortarName = gu_cortarName[i]
    #print(dong_result)
    try:
        for j in range(len(dong_result)):
            # 동 별로 빌라 & 주택 가져오기
            url = 'https://new.land.naver.com/houses?ms={0},{1},16&a=DDDGG:JWJT:SGJT:HOJT:VL&b=A1&e=RETAIL'.format(dong_centerLat[j],dong_centerLon[j])
            driver.get(url)
            # 매물 유무 확인
            Full_Xpath = '/html/body/div[2]/div/section/div[2]/div[1]/div/div[2]/div/div[2]'
            element = driver.find_element("xpath", Full_Xpath)
            time.sleep(2)

            if element.text == '해당되는 매물이 없습니다.' or element.text == '':
                print('매물 없음')
            else:
                print('매물 있음')

                # 매물 클릭하기 전 탭 개수
                tab_cnt = len(driver.window_handles)
                for k in range(1,50):
                    Full_Xpath = '/html/body/div[2]/div/section/div[2]/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div[{0}]/div'.format(k)
                    elements = driver.find_elements('xpath',Full_Xpath)
                    time.sleep(2)
                    if len(elements) > 0:
                        try:
                            elem = driver.find_element('xpath',Full_Xpath)
                            driver.execute_script("arguments[0].scrollIntoView();", elem)
                            # 매물 클릭
                            elem.click()
                        except StaleElementReferenceException:
                            elem = driver.find_element('xpath',Full_Xpath)
                            driver.execute_script("arguments[0].scrollIntoView();", elem)
                            # 매물 클릭
                            elem.click()
                        except Exception:
                            continue

                        time.sleep(2)
                        building_dict = dict()
                        # 매물 클릭한 후 탭 개수
                        click_tab_cnt = len(driver.window_handles)

                        # 클릭 후 새로운 탭이 열렸을 때
                        if tab_cnt < click_tab_cnt:
                            time.sleep(4)
                            driver.switch_to.window(driver.window_handles[1])
                            driver.close()
                            driver.switch_to.window(driver.window_handles[0])
                            # i 번째에 있는 매물의 '네이버에서 보기' 버튼 누르기
                            try:
                                full_xpath = '/html/body/div[2]/div/section/div[2]/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div[{0}]/div/div[2]/a'.format(k)
                                elem = driver.find_element('xpath',full_xpath)
                                elem.click()
                            except StaleElementReferenceException:
                                full_xpath = '/html/body/div[2]/div/section/div[2]/div[1]/div/div[2]/div/div[2]/div/div/div[1]/div[{0}]/div/div[2]/a'.format(k)
                                elem = driver.find_element('xpath',full_xpath)
                                elem.click()
                            except Exception:
                                continue

                            time.sleep(2)

                        # 건물의 사진이 없는 경우
                        try:
                            # 매물정보 탭에서 매물정보의 개수 구하기
                            # tbody 내부의 tr 요소들을 가져오기 위한 full XPath
                            tr_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr'
                                        
                            # tr 요소들을 찾아오기
                            tr_elements = driver.find_elements('xpath',tr_xpath)
                            # tr 요소의 개수 출력
                            tr_len = len(tr_elements)

                            for tr_row in range(1, tr_len+1):
                                th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th'.format(tr_row)
                                th_elements = driver.find_elements('xpath',th_xpath)
                                th_len = len(th_elements)
                                if th_len == 1:
                                    th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th'.format(tr_row)
                                    td_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/td'.format(tr_row)
                                    th = driver.find_element('xpath',th_xpath).text
                                    td = driver.find_element('xpath',td_xpath).text
                                    building_dict[th] = td
                                else:
                                    for th_row in range(1,th_len+1):
                                        th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th[{1}]'.format(tr_row, th_row)
                                        td_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/td[{1}]'.format(tr_row, th_row)
                                        th = driver.find_element('xpath',th_xpath).text
                                        td = driver.find_element('xpath',td_xpath).text
                                        building_dict[th] = td
                            # DataFrame에 매물정보 저장
                            temp_df = pd.DataFrame([building_dict])

                            # 건물명과 가격 가져오기
                            building_name_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[2]/h4/strong'                 
                            building_price_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[3]/span[2]'
                            
                            temp_df.loc[0,'building_name'] = driver.find_element('xpath',building_name_xpath).text
                            temp_df.loc[0,'price'] = driver.find_element('xpath',building_price_xpath).text

                            # 구, 동에 대한 정보 추가
                            temp_df.loc[0,'gu'] = pre_gu_cortarName
                            temp_df.loc[0,'dong'] = dong_cortarName[j]
                            temp_df.loc[0,'latitude'] = dong_centerLat[j]
                            temp_df.loc[0,'longitude'] = dong_centerLon[j]
                            
                            # 집 종류
                            building_house_type = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[1]/div[1]/div[1]/span[3]'
                            temp_df.loc[0,'house_type'] = driver.find_element('xpath',building_house_type).text

                        # 건물의 사진이 있는 경우
                        except NoSuchElementException:
                            # 매물정보 탭에서 매물정보의 개수 구하기
                            # tbody 내부의 tr 요소들을 가져오기 위한 full XPath
                            tr_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr'


                            # tr 요소들을 찾아오기
                            tr_elements = driver.find_elements('xpath',tr_xpath)
                            # tr 요소의 개수 출력
                            tr_len = len(tr_elements)

                            for tr_row in range(1, tr_len+1):
                                th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th'.format(tr_row)
                                th_elements = driver.find_elements('xpath',th_xpath)
                                th_len = len(th_elements)
                                if th_len == 1:
                                    th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th'.format(tr_row)
                                    td_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/td'.format(tr_row)
                                    th = driver.find_element('xpath',th_xpath).text
                                    td = driver.find_element('xpath',td_xpath).text
                                    building_dict[th] = td
                                else:
                                    for th_row in range(1,th_len+1):
                                        th_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/th[{1}]'.format(tr_row, th_row)
                                        td_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[3]/div[1]/table/tbody/tr[{0}]/td[{1}]'.format(tr_row, th_row)
                                        th = driver.find_element('xpath',th_xpath).text
                                        td = driver.find_element('xpath',td_xpath).text
                                        building_dict[th] = td
                            # DataFrame에 매물정보 저장
                            temp_df = pd.DataFrame([building_dict])

                            # 건물명과 가격 가져오기
                            building_name_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[2]/h4/strong'                                       
                            building_price_xpath = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[3]/span[2]'
                                                    
                            temp_df.loc[0,'building_name'] = driver.find_element('xpath',building_name_xpath).text
                            temp_df.loc[0,'price'] = driver.find_element('xpath',building_price_xpath).text

                            # 구, 동에 대한 정보 추가
                            temp_df.loc[0,'gu'] = pre_gu_cortarName
                            temp_df.loc[0,'dong'] = dong_cortarName[j]
                            temp_df.loc[0,'latitude'] = dong_centerLat[j]
                            temp_df.loc[0,'longitude'] = dong_centerLon[j]
                            
                            # 집 종류
                            building_house_type = '/html/body/div[2]/div/section/div[2]/div[2]/div/div[2]/div[2]/div[1]/div[1]/span[3]'
                            temp_df.loc[0,'house_type'] = driver.find_element('xpath',building_house_type).text
                        except Exception:
                            temp_df = pd.DataFrame()

                        total_df = pd.concat([total_df,temp_df],ignore_index = True)
                        print(total_df.iloc[-1]['building_name'])
                    else:
                        break
                time.sleep(3)
    except Exception:
        print('except')
        continue

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

매물 있음
다가구
빌라 누리찬
다가구
빌라 누리찬
빌라 누리찬빌라
빌라
빌라 삼성베스트빌 A동
빌라 개포런던빌
단독
단독
빌라 개포런던빌
빌라
빌라
빌라 개발호재모아타운대지지분7
빌라
빌라
빌라 서초빌라 1동
빌라
빌라
빌라
빌라
빌라 개포상지리츠빌
빌라
빌라
빌라 뉴아트빌라 1동
빌라
빌라 1동


  4%|███▏                                                                            | 1/25 [03:30<1:24:15, 210.64s/it]

except
매물 없음
매물 있음
빌라 연희펠리시티
빌라
빌라
단독
다가구
단독
빌라
빌라
단독
상가주택 1
상가주택
빌라
빌라 고덕힐하우스
빌라 고덕힐하우스
빌라 하우진
빌라
빌라 하우진
단독
빌라 하우진
빌라
빌라 하우진
빌라
단독
단독 단독 다가구
단독


  8%|██████▍                                                                         | 2/25 [06:29<1:13:39, 192.16s/it]

...

except
매물 없음
매물 있음


 96%|███████████████████████████████████████████████████████████████████████████▊   | 24/25 [1:18:46<04:05, 246.00s/it]

except
매물 있음
빌라
빌라
빌라
빌라
빌라
빌라
상가주택
빌라
빌라
빌라
빌라
단독
빌라
빌라
단독
상가주택
빌라
빌라
상가주택
빌라
단독
빌라
빌라 스타빌
빌라
빌라
빌라
빌라
빌라
빌라
빌라 우림팰리스
빌라 브라운캐슬
빌라 브라운캐슬 에이동
빌라
단독
빌라
빌라
연립 신일연립 나 동


100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:23:20<00:00, 200.02s/it]

except


In [11]:
total_df.to_csv('./data/naver_selenium_villa_info.csv')

In [12]:
total_df

,building_name,price,gu,dong,latitude,longitude,house_type,소재지,매물특징,공급/전용면적,...,중개사,대지/연면적,건축/전용면적,지상층/지하층,복층여부,재건축/재개발,월관리비,관리비 포함,직거래 정보,준공인가일
0,다가구,"11억 5,000",강남구,개포동,37.482968,127.0634,단독/다가구,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,빌라 누리찬,"12억 5,000",강남구,개포동,37.482968,127.0634,빌라,서울시 강남구 개포동 169-8,"복층세대, 입지와 사업성 모두 만족하는 강남 재개발지역투자",35.15㎡/28.49㎡(전용률81%),...,개포1번지공인중개사사무소톡톡문의길찾기\n대표이관우등록번호\n소재지\n서울특별시 강남...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,다가구,40억,강남구,개포동,37.482968,127.0634,단독/다가구,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,빌라 누리찬,11억,강남구,개포동,37.482968,127.0634,빌라,서울시 강남구 개포동 169-8,"복층세대, 입지와 사업성 모두 만족하는 강남 재개발지역투자",30.84㎡/25㎡(전용률81%),...,개포1번지공인중개사사무소톡톡문의길찾기\n대표이관우등록번호\n소재지\n서울특별시 강남...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,빌라 누리찬빌라,"8억 4,000",강남구,개포동,37.482968,127.0634,빌라,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
688,빌라,"4억 6,700",중랑구,망우동,37.600899,127.1073,빌라,서울시 중랑구 망우동 396-41,3룸 큰거실 우림시장인근 주차 좋아요,54.34㎡/45.96㎡(전용률85%),...,우남공인중개사사무소길찾기\n대표노지훈등록번호\n소재지\n서울특별시 중랑구 봉화산로5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
689,단독,8억,중랑구,망우동,37.600899,127.1073,단독/다가구,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
690,빌라,"4억 7,500",중랑구,망우동,37.600899,127.1073,빌라,서울시 중랑구 망우동 396-35,망우동 신축빌라 3룸 매매 첫입주,63.32㎡/55.53㎡(전용률88%),...,우남공인중개사사무소길찾기\n대표노지훈등록번호\n소재지\n서울특별시 중랑구 봉화산로5...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
691,빌라,"3억 8,800",중랑구,망우동,37.600899,127.1073,빌라,서울시 중랑구 망우동 462-19,방3개 최저가 구조 크기 비교불가 모아타운예정 실거주 투자모두추천,58.64㎡/49.59㎡(전용률85%),...,골드부동산길찾기\n대표한진희등록번호\n소재지\n서울특별시 중랑구 망우동 43-0\n...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
